In [21]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import TensorDataset, DataLoader, Dataset, random_split
import json
import requests
from PIL import Image
from io import BytesIO
import numpy as np  
import matplotlib.pyplot as plt
import os
import pandas as pd
from tqdm import tqdm

<h3><i>Download images</i></h3>

In [2]:
df = pd.read_csv('preprocess/pizza_data_final.csv', index_col='id')

C:\Users\DuyAnh\AppData\Local\Temp\ipykernel_7000\1442764996.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('preprocess/pizza_data_final.csv', index_col='id')


In [3]:
all_error_list = ['Bánh không tròn | Distorted shape',
 'Cháy | Baking - Burnt',
 'Viền k đều | Edge - Uneven',
 'Bánh bé | Size - Too small',
 'Thiếu bóng | too few balls',
 'Không đốm | Baking - Does not have leopard-spotting',
 'Màu nhạt | Baking - Pale',
 'Viền nhỏ | Edge - Too small',
 'Viền to | Edge - Too big',
 'Nở viền không đủ | edge pizza is not enough swollen',
 'Viền thấp | Edge - Too low',
 'Phô mai cao | Topping - Cheese too high * with a core*',
 'Topping - Không đúng | incorrect portioning',
 'Không cân Topping - Not even half and half',
 'Sốt trên mép | Topping - Sauce covering on the edge',
 'Lên men thiếu | Fermentation - Lack fermentation',
 'Quá theo viền | Topping - Too strong shaping the edge',
 'Trộn lẫn | Topping - Bended',
 'Quá tập trung | Topping - Topping too centered',
 'Topping - Không đều | Not even',
 'Lên men quá nhiều | Fermentation - Over fermentation',
 'Không hình tròn | Topping - Not circled',
 'Bánh lớn | Size - Too big']
# tổng cộng có 23 loại lỗi

In [4]:
df = df.iloc[70000:72000]

In [5]:
len(df)

2000

In [6]:
queue_download_image = [] 
for image_id, image_url in zip(df.index, df['image_url']):
    queue_download_image.append((image_id, image_url))
len(queue_download_image)

2000

In [9]:
# # Download
# data_path = 'data/pizza_test_predict'
# for id, url in tqdm(queue_download_image):
#     response = requests.get(url)
#     image_data = BytesIO(response.content)
#     image = Image.open(image_data)
#     image_file_name = os.path.join(data_path, f"{id}.jpg")
#     image.save(image_file_name)
#     if response.status_code == 200:
#         pass
#     else:
#         print(f"Failed to download {image_url}")

100%|██████████| 2000/2000 [17:03<00:00,  1.95it/s]


In [7]:
dict_labels = {}   # lưu nhãn theo dạng [0, 1, 0, 1, 1, 0, 0, 0, ...]
for id, _ in tqdm(queue_download_image):
    dict_labels[id] = []
    for error in all_error_list:
        dict_labels[id].append(df.at[id, error])

100%|██████████| 2000/2000 [00:00<00:00, 10547.75it/s]


In [8]:
# chuẩn bị image_paths, labels
path_to_image = 'data/pizza_test_predict/'
image_paths = []
labels = []
for id, _ in tqdm(queue_download_image):
    image_paths.append(path_to_image + f'{id}.jpg')
    labels.append(torch.tensor(dict_labels[id], dtype = torch.float))

100%|██████████| 2000/2000 [00:00<00:00, 63311.20it/s]


In [9]:
labels = [label_tensor[:20] for label_tensor in labels]    # bỏ 3 nhãn cuối
len(labels)

2000

In [10]:
transform = transforms.Compose([    # hàm để biến đổi, chuẩn hóa ảnh và đưa về tensor
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [11]:
class CustomImageDataset(Dataset):
    def __init__(self, image_paths, labels, transforms = None):
        self.image_paths = image_paths
        self.labels = labels
        self.transforms = transforms
    
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path)
        label = self.labels[idx]
        if self.transforms:
            image = self.transforms(image)
        return image, label

In [18]:
dataset = CustomImageDataset(image_paths=image_paths, labels=labels, transforms=transform)

data_loader = DataLoader(dataset, batch_size = 64, shuffle = True)

2000


In [28]:
from sklearn.metrics import precision_score, recall_score, average_precision_score

def calculate_precision_recall(y_pred, y_true):
    precision = precision_score(y_true, y_pred, average=None)
    recall = recall_score(y_true, y_pred, average=None)
    return precision, recall

def calculate_mAP(y_pred, y_true):
    AP_scores = [average_precision_score(y_true[:, i], y_pred[:, i]) for i in range(20)]
    mAP_score = np.mean(AP_scores)
    return mAP_score, AP_scores

In [29]:
model = torchvision.models.resnet50(pretrained = True)
num_classes = 20
in_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(512, num_classes),
    nn.Sigmoid()
)

for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True
for param in model.layer4.parameters():
    param.requires_grad = True

C:\Users\DuyAnh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\DuyAnh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [30]:
model.load_state_dict(torch.load('model/best_model_ver2.pth'))

<All keys matched successfully>

In [31]:
model.eval()
prediction_labels = []
round_pred_lables = []
true_labels = []
with torch.inference_mode():
    for inputs, label in tqdm(data_loader):
        outputs = model(inputs)
        round_outputs = torch.round(outputs)
        prediction_labels.extend(outputs.detach().numpy())
        true_labels.extend(label.detach().numpy())
        round_pred_lables.extend(round_outputs.detach().numpy())
    prediction_labels = np.vstack(prediction_labels)
    true_labels  = np.vstack(true_labels)
    round_pred_lables = np.vstack(round_pred_lables)

    mAP, AP = calculate_mAP(prediction_labels, true_labels)
    precision, recall = calculate_precision_recall(round_pred_lables, true_labels)
    print(mAP)
    print([f'{ap:.3f}' for ap in AP])
    print([f'{p:.3f}' for p in precision])
    print([f'{r:.3f}' for r in recall])


100%|██████████| 32/32 [03:58<00:00,  7.46s/it]

0.5699699225023347
['0.859', '0.687', '0.645', '0.573', '0.822', '0.828', '0.693', '0.632', '0.735', '0.609', '0.380', '0.696', '0.597', '0.481', '0.441', '0.218', '0.188', '0.369', '0.686', '0.261']
['0.811', '0.715', '0.722', '0.662', '0.718', '0.753', '0.654', '0.594', '0.725', '0.728', '0.521', '0.684', '0.692', '0.640', '0.373', '0.132', '0.184', '0.284', '0.255', '0.231']
['0.696', '0.509', '0.352', '0.422', '0.763', '0.738', '0.570', '0.552', '0.525', '0.324', '0.487', '0.580', '0.364', '0.163', '0.604', '0.500', '0.348', '0.541', '0.897', '0.610']
